In [3]:
import os
from bs4 import BeautifulSoup
import re
from collections import defaultdict

def combine_html_reports(input_directory, output_file):
    """
    Combines multiple HTML files into a single interactive HTML page,
    organizing raw and non-raw versions for each cell type.
    
    Args:
        input_directory (str): Directory containing HTML files
        output_file (str): Path to save the combined HTML file
    """
    # Get all HTML files and organize them by cell type
    html_files = [f for f in os.listdir(input_directory) if f.endswith('.html')]
    cell_types = defaultdict(dict)
    
    for file in html_files:
        is_raw = '_raw.html' in file
        # Extract cell type name by removing '_raw.html' or '.html'
        cell_type = file.replace('_raw.html', '').replace('.html', '')
        if is_raw:
            cell_types[cell_type]['raw'] = file
        else:
            cell_types[cell_type]['normal'] = file
    
    # Create the base structure for the combined HTML
    combined_html = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>Cell Type Reports</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                margin: 0;
                padding: 20px;
                background-color: #f5f5f5;
            }
            .nav-container {
                background-color: #fff;
                padding: 15px;
                border-radius: 5px;
                margin-bottom: 20px;
                box-shadow: 0 2px 4px rgba(0,0,0,0.1);
                display: flex;
                flex-wrap: wrap;
                gap: 10px;
            }
            .tab-button {
                padding: 10px 20px;
                border: none;
                border-radius: 4px;
                background-color: #f0f0f0;
                cursor: pointer;
                transition: background-color 0.3s;
            }
            .tab-button:hover {
                background-color: #e0e0e0;
            }
            .tab-button.active {
                background-color: #007bff;
                color: white;
            }
            .content-container {
                background-color: #fff;
                padding: 20px;
                border-radius: 5px;
                box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            }
            .tab-content {
                display: none;
            }
            .tab-content.active {
                display: block;
            }
            .toggle-container {
                display: flex;
                align-items: center;
                margin-bottom: 15px;
                padding: 10px;
                background-color: #f8f9fa;
                border-radius: 4px;
            }
            .toggle-switch {
                position: relative;
                display: inline-block;
                width: 60px;
                height: 34px;
                margin-right: 10px;
            }
            .toggle-switch input {
                opacity: 0;
                width: 0;
                height: 0;
            }
            .slider {
                position: absolute;
                cursor: pointer;
                top: 0;
                left: 0;
                right: 0;
                bottom: 0;
                background-color: #ccc;
                transition: .4s;
                border-radius: 34px;
            }
            .slider:before {
                position: absolute;
                content: "";
                height: 26px;
                width: 26px;
                left: 4px;
                bottom: 4px;
                background-color: white;
                transition: .4s;
                border-radius: 50%;
            }
            input:checked + .slider {
                background-color: #2196F3;
            }
            input:checked + .slider:before {
                transform: translateX(26px);
            }
            .toggle-label {
                font-weight: bold;
            }
        </style>
    </head>
    <body>
        <div class="nav-container" id="tabContainer"></div>
        <div class="content-container">
            <div id="contentContainer"></div>
        </div>
        
        <script>
            function openTab(evt, tabName) {
                var tabcontent = document.getElementsByClassName("tab-content");
                for (var i = 0; i < tabcontent.length; i++) {
                    tabcontent[i].classList.remove("active");
                }
                
                var tablinks = document.getElementsByClassName("tab-button");
                for (var i = 0; i < tablinks.length; i++) {
                    tablinks[i].classList.remove("active");
                }
                
                document.getElementById(tabName).classList.add("active");
                evt.currentTarget.classList.add("active");
            }
            
            function toggleRaw(cellType) {
                const normalContent = document.getElementById(`${cellType}_normal`);
                const rawContent = document.getElementById(`${cellType}_raw`);
                
                if (normalContent && rawContent) {
                    if (normalContent.style.display === "none") {
                        normalContent.style.display = "block";
                        rawContent.style.display = "none";
                    } else {
                        normalContent.style.display = "none";
                        rawContent.style.display = "block";
                    }
                }
            }
        </script>
    </body>
    </html>
    """
    
    soup = BeautifulSoup(combined_html, 'html.parser')
    
    # Create containers for tabs and content
    tab_container = soup.find(id="tabContainer")
    content_container = soup.find(id="contentContainer")
    
    # Process each cell type
    for i, (cell_type, versions) in enumerate(sorted(cell_types.items())):
        # Create tab button
        button = soup.new_tag("button")
        button["class"] = "tab-button"
        button["onclick"] = f"openTab(event, 'tab_{i}')"
        if i == 0:
            button["class"] = "tab-button active"
        button.string = cell_type
        tab_container.append(button)
        
        # Create main content div for this cell type
        main_content_div = soup.new_tag("div")
        main_content_div["class"] = "tab-content"
        main_content_div["id"] = f"tab_{i}"
        if i == 0:
            main_content_div["class"] = "tab-content active"
        
        # Add toggle switch
        toggle_container = soup.new_tag("div")
        toggle_container["class"] = "toggle-container"
        
        toggle_switch = soup.new_tag("label")
        toggle_switch["class"] = "toggle-switch"
        
        toggle_input = soup.new_tag("input")
        toggle_input["type"] = "checkbox"
        toggle_input["onchange"] = f"toggleRaw('{cell_type}')"
        
        toggle_slider = soup.new_tag("span")
        toggle_slider["class"] = "slider"
        
        toggle_label = soup.new_tag("span")
        toggle_label["class"] = "toggle-label"
        toggle_label.string = "Show Raw Data"
        
        toggle_switch.append(toggle_input)
        toggle_switch.append(toggle_slider)
        toggle_container.append(toggle_switch)
        toggle_container.append(toggle_label)
        main_content_div.append(toggle_container)
        
        # Add normal and raw content
        for version_type in ['normal', 'raw']:
            if version_type in versions:
                file_path = os.path.join(input_directory, versions[version_type])
                with open(file_path, 'r', encoding='utf-8') as f:
                    file_content = f.read()
                
                content_div = soup.new_tag("div")
                content_div["id"] = f"{cell_type}_{version_type}"
                content_div["style"] = "display: block" if version_type == 'normal' else "display: none"
                
                file_soup = BeautifulSoup(file_content, 'html.parser')
                body_content = file_soup.find('body')
                if body_content:
                    content_div.append(body_content)
                else:
                    content_div.append(file_soup)
                
                main_content_div.append(content_div)
        
        content_container.append(main_content_div)
    
    # Save the combined HTML
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(str(soup.prettify()))

In [ ]:
C:\Users\ellio\OneDrive - UW-Madison\cellgpt_final_folder\paper_results\immune_projecttils\score_nonmodel

In [7]:
if __name__ == "__main__":
    combine_html_reports("C:/Users/ellio/OneDrive - UW-Madison/cellgpt_final_folder/paper_results/immune_projecttils/score_nonmodel", "C:/Users/ellio/OneDrive - UW-Madison/cellgpt_final_folder/paper_results/immune_projecttils/combined_report_nonmodel.html")

In [6]:
# Example usage
if __name__ == "__main__":
    combine_html_reports("C:/Users/ellio/OneDrive - UW-Madison/cellgpt_final_folder/paper_results/immune_projecttils/score_report", "C:/Users/ellio/OneDrive - UW-Madison/cellgpt_final_folder/paper_results/immune_projecttils/combined_report2.html")

In [9]:

# Example usage
if __name__ == "__main__":
    combine_html_reports("C:/Users/ellio/OneDrive - UW-Madison/cellgpt_final_folder/paper_results/immune_projecttils/score_nonmodel", "C:/Users/ellio/OneDrive - UW-Madison/cellgpt_final_folder/paper_results/immune_projecttils/combined_report2_non.html")

In [8]:
if __name__ == "__main__":
    combine_html_reports("C:/Users/ellio/Downloads/annotation+boost", "C:/Users/ellio/Downloads/annotation+boost/combined_report_otherthanMCA.html")



In [5]:
def combine_html_reports(input_directory, output_file):
    """
    Combines multiple HTML files into a single interactive HTML page,
    organizing raw and non-raw versions for each cell type.
    """
    # Get all HTML files and organize them by cell type
    html_files = [f for f in os.listdir(input_directory) if f.endswith('.html')]
    cell_types = defaultdict(dict)
    
    for file in html_files:
        is_raw = '_raw.html' in file
        cell_type = file.replace('_raw.html', '').replace('.html', '')
        if is_raw:
            cell_types[cell_type]['raw'] = file
        else:
            cell_types[cell_type]['normal'] = file
    
    # Create the base structure
    combined_html = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>Cell Type Reports</title>
        <style>
            .nav-container {
                background-color: #fff;
                padding: 15px;
                border-radius: 5px;
                margin-bottom: 20px;
                box-shadow: 0 2px 4px rgba(0,0,0,0.1);
                display: flex;
                flex-wrap: wrap;
                gap: 10px;
                position: sticky;
                top: 0;
                z-index: 1000;
            }
            .tab-button {
                padding: 10px 20px;
                border: none;
                border-radius: 4px;
                background-color: #f0f0f0;
                cursor: pointer;
                transition: background-color 0.3s;
            }
            .tab-button:hover {
                background-color: #e0e0e0;
            }
            .tab-button.active {
                background-color: #007bff;
                color: white;
            }
            .tab-content {
                display: none;
            }
            .tab-content.active {
                display: block;
            }
            .toggle-container {
                position: sticky;
                top: 70px;
                background: white;
                z-index: 999;
                padding: 10px;
                margin-bottom: 10px;
                border-bottom: 1px solid #eee;
            }
        </style>
    </head>
    <body>
        <div class="nav-container" id="tabContainer"></div>
        <div id="contentContainer"></div>
        
        <script>
            function openTab(evt, tabName) {
                var tabcontent = document.getElementsByClassName("tab-content");
                for (var i = 0; i < tabcontent.length; i++) {
                    tabcontent[i].classList.remove("active");
                }
                
                var tablinks = document.getElementsByClassName("tab-button");
                for (var i = 0; i < tablinks.length; i++) {
                    tablinks[i].classList.remove("active");
                }
                
                document.getElementById(tabName).classList.add("active");
                evt.currentTarget.classList.add("active");
            }
            
            function toggleRaw(cellType) {
                const normalContent = document.getElementById(`${cellType}_normal`);
                const rawContent = document.getElementById(`${cellType}_raw`);
                
                if (normalContent && rawContent) {
                    if (normalContent.style.display === "none") {
                        normalContent.style.display = "block";
                        rawContent.style.display = "none";
                    } else {
                        normalContent.style.display = "none";
                        rawContent.style.display = "block";
                    }
                }
            }
        </script>
    </body>
    </html>
    """
    
    soup = BeautifulSoup(combined_html, 'html.parser')
    
    # Create containers for tabs and content
    tab_container = soup.find(id="tabContainer")
    content_container = soup.find(id="contentContainer")
    
    # Process each cell type
    for i, (cell_type, versions) in enumerate(sorted(cell_types.items())):
        # Create tab button
        button = soup.new_tag("button")
        button["class"] = "tab-button"
        button["onclick"] = f"openTab(event, 'tab_{i}')"
        if i == 0:
            button["class"] = "tab-button active"
        button.string = cell_type
        tab_container.append(button)
        
        # Create content div
        main_content_div = soup.new_tag("div")
        main_content_div["class"] = "tab-content"
        main_content_div["id"] = f"tab_{i}"
        if i == 0:
            main_content_div["class"] = "tab-content active"
        
        # Add toggle button
        toggle_html = f"""
        <div class="toggle-container">
            <label>
                <input type="checkbox" onchange="toggleRaw('{cell_type}')">
                Show Raw Data
            </label>
        </div>
        """
        main_content_div.append(BeautifulSoup(toggle_html, 'html.parser'))
        
        # Add normal and raw content
        for version_type in ['normal', 'raw']:
            if version_type in versions:
                file_path = os.path.join(input_directory, versions[version_type])
                with open(file_path, 'r', encoding='utf-8') as f:
                    file_content = f.read()
                
                # Create a container for this version
                content_div = soup.new_tag("div")
                content_div["id"] = f"{cell_type}_{version_type}"
                content_div["style"] = "display: block" if version_type == 'normal' else "display: none"
                
                # Insert the entire HTML content
                content_div.append(BeautifulSoup(file_content, 'html.parser'))
                main_content_div.append(content_div)
        
        content_container.append(main_content_div)
    
    # Save the combined HTML
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(str(soup))
